## Isochrone Node
Wat is een [isochrone](https://en.wiktionary.org/wiki/isochrone)?

1 -> 1

Documentatie Open Route Service (ORS) Python Client: http://openrouteservice-py.readthedocs.io/en/latest/  
Documentatie ORS Isochrones: https://openrouteservice.org/documentation/#/reference/isochrones

*Vereiste:* `input_table` met `wkt`-kolom met punten.

*Note:* Pas de time range (`drivingTime`) aan.

In [2]:
###
### USER DEFINED
###

# Driving time in minutes
drivingTime = 10

In [3]:
###
### HEADER
###
import pandas as pd
from shapely.geometry import Polygon, Point
from shapely import wkt as WKT
import openrouteservice
from openrouteservice.isochrones import isochrones

# initialize ORS with the github ab-os account
ORS = openrouteservice.Client(key='58d904a497c67e00015b45fc45f368a7463145e7be57c7563a900810')

# define the get isochrone function from WKT point
def getIsochrone(point, drivingTime): 
    # extract long lat coordinates from WKT
    longLat = wktToLongLat(point)
    # create the ORS isochrone from te long lat coords
    isochrone = isochrones(ORS, locations=longLat, intervals=[drivingTime*60], segments=[])
    # convert XML (eigenlijk dictionary) to Polygon to WKT
    return WKT.dumps(Polygon([(xy[0], xy[1]) for xy in isochrone['features'][0]['geometry']['coordinates'][0]]), trim=True)

# from point (WKT) to longLat (tuple)
def wktToLongLat(point):
    # eerst van WKT naar geometry
    geom = WKT.loads(point)
    # dan naar tuple (long, lat)
    return (geom.x, geom.y)

In [4]:
###
### ISOCHRONE
###
    
# start off by copying the input table
print('Shape input_table:', input_table.shape)
output_table = input_table.copy()

# get isochrones
output_table['wkt'] = output_table.wkt.apply(getIsochrone, args=(drivingTime,))

# output
print('Shape output_table:', output_table.shape)

Shape input_table: (2, 2)
Shape output_table: (2, 2)


In [5]:
###
### FOOTER
###
print('Preview output_table (first 5 rows):')
try:
    # try pretty print in Jupyter Notebook
    display(output_table.head())
except NameError:
    print(output_table.head())

Preview output_table (first 5 rows):


,locatie,wkt
0,EIFFEL Arnhem,"POLYGON ((5.293363 52.115799, 5.294052 52.1122..."
1,EIFFEL Eindhoven,"POLYGON ((5.308315 51.492356, 5.309227 51.4921..."


## Load some input data to test the cells above

In [1]:
###
### SOURCE - ONLY FOR PREPARATION
###
import pandas as pd
input_table = pd.DataFrame(data={'locatie': ['EIFFEL Arnhem', 'EIFFEL Eindhoven'],\
                                 'wkt': ['POINT (5.38782 52.15611)', 'POINT (5.46122 51.44491)']})

# check
input_table

,locatie,wkt
0,EIFFEL Arnhem,POINT (5.38782 52.15611)
1,EIFFEL Eindhoven,POINT (5.46122 51.44491)
